In [1]:
from find_source import make_catalog, combine_catalogs
import glob
import pandas as pd

In [2]:
final_catalog = {}
for file in glob.glob('../data/new_250609_17:30:22/*.fits'):
    try:
        final_catalog = combine_catalogs(final_catalog, make_catalog(file))
    except:
        print(f'Try again for {file}')

In [3]:
df = pd.DataFrame.from_dict(final_catalog)
print(df)

                                                           source_1  \
field_name                                                 2232+117   
obs_date_time                                      6-10-25 11:16:38   
file_name                                        2232+117_full.fits   
beam_maj_axis                                     0d00m01.69712316s   
beam_min_axis                                     0d00m00.78837516s   
beam_pos_angle                                        99d00m22.356s   
flux_uncertainty                                           0.025114   
flux_density                                               3.225516   
coord             <SkyCoord (ICRS): (ra, dec) in deg\n    (338.1...   
internal                                                       True   

                                                           source_2  \
field_name                                                 2232+117   
obs_date_time                                      6-10-25 11:16:38   
file_